In [12]:
import matplotlib.pyplot as plt
import pandas            as pd
import scipy.stats       as stats
import seaborn           as sns
import statsmodels.api   as sm
import numpy as np

from statsmodels.formula.api     import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [13]:
f = open('LDS3 - De thi cucoi khoa K268/Samples.txt', 'r')
data = f.read()
f.close()
data = data.split("\n")
df= pd.DataFrame.from_records([r.split('\t') for r in data[1:]], columns=data[0].split('\t'))

In [14]:
# Loại bỏ các giá trị là khoảng trắng trong dataframe khi lấy dữ liệu từ file text
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        try:
            df.iloc[i,j] = int(df.iloc[i,j])
        except:
            df.iloc[i,j] = np.NAN
df.tail()

,A,B,C,D
10,78,NaN,45,NaN
11,95,NaN,55,NaN
12,93,NaN,NaN,NaN
13,88,NaN,NaN,NaN
14,110,NaN,NaN,NaN


In [15]:
k      = len(df.columns)
groups = list(df.columns) # (k column headers)

samples = []
for j in range(k):
    sample = [x for x in df[groups[j]] if pd.notnull(x)]
    samples.append(sample)

In [16]:
A= samples[0]
B= samples[1]
C= samples[2]
D= samples[3]

In [17]:
# 6.1) Với alpha = 0.05, hãy kiểm định giả thuyết H0: “Các quần thể có cùng phương sai.”

# H0: Phương sai của A, B, C, D bằng nhau
# Ha: Phương sai của A, B, C, D KHÔNG BẰNG NHAU

levene, pvalue = stats.levene(A, B, C, D)
print('Levene-statistic = %.4f, p-value = %.4f' % (levene, pvalue))

# p-value > 0.05 => Chấp nhận H0 -> A, B, C, D  có phương sai bằng nhau

Levene-statistic = 0.7978, p-value = 0.5021


In [18]:
# 6.2) Với alpha = 0.05, hãy cho kết luận về giả thuyết H0: “Các quần thể có cùng giá trị trung bình.”
# Dùng phương pháp truyền thống

# One-way ANOVA
# H0: Trung bình của A, B, C, D khác biệt không đáng kể
# Ha: Có ít nhất 1 cặp trong A, B, C, D có trung bình khác biệt đáng kể

alpha = 0.05
## Số phần tử của mỗi nhóm
sizes = np.zeros(k)
for j in range(k): 
    sizes[j] = np.size(samples[j])

## Giá trị trung bình của mỗi mẫu
means = np.zeros(k)
for j in range(k):
    means[j] = np.mean(samples[j])

## Giá trị trung bình của tất cả các mẫu
meanT = np.mean(means)

## Các đại lượng BETWEEN groups: SSB, dfB
SSB = 0
for j in range(k):
    SSB += sizes[j] * np.power((means[j] - meanT), 2)
dfB = (k - 1)
print('SSB           : %.4f' %SSB)
print('dfB           : %d' %dfB)
   
## Các đại lượng WITHIN groups: SSW, dfW
SSW = 0
for j in range(k):
    SSWj = 0
    for i in range(int(sizes[j])):
        SSWj += np.power(samples[j][i] - means[j], 2)
    SSW += SSWj
dfW = int(np.sum(sizes) - k)
print('SSW           : %.4f' %SSW)
print('dfW           : %d' %dfW)

## Trị thống kê: F statistics
F = (SSB / dfB) / (SSW / dfW)
print('F statistic   : %.4f' %F)

## Giá trị tới hạn

critical_value = stats.f.ppf(q = 1 - alpha, dfn = dfB, dfd = dfW) # xác định giá trị tới hạn
print('Critical value: %.4f' %critical_value)

if (F < critical_value):
    print('(F <  critical value) ==> Chấp nhận H0: <H0 : Trung bình của A, B, C, D khác biệt không đáng kể.>')
else:
    print('(F >= critical value) ==> Bác bỏ HO: <H0 : Trung bình của A, B, C, D khác biệt không đáng kể.>')

SSB           : 12486.4848
dfB           : 3
SSW           : 11464.5722
dfW           : 42
F statistic   : 15.2479
Critical value: 2.8270
(F >= critical value) ==> Bác bỏ HO: <H0 : Trung bình của A, B, C, D khác biệt không đáng kể.>


In [19]:
# 6.2) Với alpha = 0.05, hãy cho kết luận về giả thuyết H0: “Các quần thể có cùng giá trị trung bình.”
# Dùng hàm thống kê sẵn

# One-way ANOVA
# H0: Trung bình của A, B, C, D khác biệt không đáng kể
# Ha: Có ít nhất 1 cặp trong A, B, C, D có trung bình khác biệt đáng kể

fvalue, pvalue = stats.f_oneway(A, B, C, D)
print('F-statistic = %.4f, p-value = %.10f' %(fvalue, pvalue))

# p-value < 0.05 -> Bác bỏ giả thuyết H0
# Do đó, có ít nhất 1 cặp trong A, B, C, D có trung bình khác biệt đáng kể

F-statistic = 15.1391, p-value = 0.0000007991


In [20]:
#6.3) Nếu bác bỏ giả thuyết H0 trong câu 6.2), hãy cho biết những quần thể nào có sự khác biệt về giá trị trung bình

# Unpivot dữ liệu để kiểm định Tukey HSD

df_melt = pd.melt(df.reset_index(), id_vars = ['index'], 
                  value_vars = ['A', 'B', 'C', 'D'])

df_melt = df_melt.dropna()
df_melt['value'] = df_melt['value'].astype('int64')

In [21]:
# Kiểm định Tukey HSD

m_comp = pairwise_tukeyhsd(endog = df_melt['value'], groups = df_melt['variable'], alpha = 0.05)
print(m_comp)

# Ngoại trừ A & D, B & C, tất cả các so sánh cặp khác đều bác bỏ H0 và chỉ ra sự khác biệt đáng kể về mặt thống kê.

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     A      B -35.3667  0.001 -53.4103  -17.323   True
     A      C -36.9167  0.001 -54.0344 -19.7989   True
     A      D -11.7778 0.3415 -30.4132   6.8577  False
     B      C    -1.55    0.9 -20.4744  17.3744  False
     B      D  23.5889 0.0171   3.2814  43.8964   True
     C      D  25.1389 0.0068   5.6495  44.6283   True
------------------------------------------------------
